In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
import numpy as np
import pandas as pd 

In [4]:
import sys
sys.path.append('../../../')

In [5]:
import matplotlib.pyplot as plt 

In [6]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [7]:
from src.utils.dates import get_nyse_business_dates

In [8]:
from research.metrics import get_features_responses_correlation
from research.plots import plot_bin_summary_of_xy

In [9]:
from src.settings import Settings

In [10]:
from src.utils.io.read import PreprocessedDataReader

In [11]:
from src.preprocessing.base import FutureTicker

In [12]:
pdr = PreprocessedDataReader(Settings.historical.paths.PREPROCESSED_DATA_PATH)                    

In [13]:
dataset = pdr.read_dataset(ticker=FutureTicker.WTI)

In [14]:
dataset.shape

(813, 63)

In [15]:
dataset.tail(2)

,tradeDate,Name,Commercial_NetPosition,CommercialLongPosition,CommercialShortPosition,ManagedMoney_NetPosition,ManagedMoney_LongPosition,ManagedMoney_ShortPosition,Commercial_NetPosition_change,prior_report_Commercial_NetPosition_change,...,forward_CommercialShortPosition_to_openinterest_change,ManagedMoney_NetPosition_to_openinterest_change,prior_report_ManagedMoney_NetPosition_to_openinterest_change,forward_ManagedMoney_NetPosition_to_openinterest_change,ManagedMoney_LongPosition_to_openinterest_change,prior_report_ManagedMoney_LongPosition_to_openinterest_change,forward_ManagedMoney_LongPosition_to_openinterest_change,ManagedMoney_ShortPosition_to_openinterest_change,prior_report_ManagedMoney_ShortPosition_to_openinterest_change,forward_ManagedMoney_ShortPosition_to_openinterest_change
811,2025-07-22,CL,355309.0,1179861.0,824552.0,208836.0,406472.0,197636.0,-13379.0,31871.0,...,-0.010335,-0.001610,-0.027664,-0.001536,0.006267,-0.021889,0.00081,0.007877,0.005775,0.002346
812,2025-07-29,CL,362436.0,1172427.0,809991.0,207342.0,411275.0,203933.0,7127.0,-13379.0,...,NaN,-0.001536,-0.001610,NaN,0.000810,0.006267,NaN,0.002346,0.007877,NaN


In [16]:
dataset.columns.tolist()

['tradeDate',
 'Name',
 'Commercial_NetPosition',
 'CommercialLongPosition',
 'CommercialShortPosition',
 'ManagedMoney_NetPosition',
 'ManagedMoney_LongPosition',
 'ManagedMoney_ShortPosition',
 'Commercial_NetPosition_change',
 'prior_report_Commercial_NetPosition_change',
 'CommercialLongPosition_change',
 'prior_report_CommercialLongPosition_change',
 'CommercialShortPosition_change',
 'prior_report_CommercialShortPosition_change',
 'ManagedMoney_NetPosition_change',
 'prior_report_ManagedMoney_NetPosition_change',
 'ManagedMoney_LongPosition_change',
 'prior_report_ManagedMoney_LongPosition_change',
 'ManagedMoney_ShortPosition_change',
 'prior_report_ManagedMoney_ShortPosition_change',
 'F1_RolledPrice',
 'F2_RolledPrice',
 'F3_RolledPrice',
 'F1_RolledPrice_rolling_20D_volatility',
 'F2_RolledPrice_rolling_20D_volatility',
 'F3_RolledPrice_rolling_20D_volatility',
 'SyntheticF1MinusF2_RolledPrice',
 'prior_report_SyntheticF1MinusF2_RolledPrice_change',
 'prior_cumulative_5D_F1_V

In [17]:
dataset['Commercial_NetPosition'].describe()

count       813.000000
mean     -11247.533825
std      156021.927084
min     -315658.000000
25%     -146916.000000
50%       13553.000000
75%       95631.000000
max      368688.000000
Name: Commercial_NetPosition, dtype: float64

In [18]:
dataset['ManagedMoney_NetPosition'].describe()

count       813.000000
mean     405093.035670
std      162420.952528
min      100633.000000
25%      273760.000000
50%      366026.000000
75%      518079.000000
max      848584.000000
Name: ManagedMoney_NetPosition, dtype: float64

In [19]:
np.sum(dataset['ManagedMoney_LongPosition'] <= dataset['ManagedMoney_ShortPosition'])

0

In [20]:
dataset['tradeDate'] = pd.to_datetime(dataset['tradeDate']).dt.date

In [21]:
dataset.sort_values(by = 'tradeDate', ascending = True, inplace=True)

In [22]:
for name in [ 'prior_cumulative_5D_F1_Volume',
                 'prior_cumulative_5D_F2_Volume',
                 'prior_cumulative_5D_F1MinusF2_Volume' ,
            'F1_RolledPrice',
 'F2_RolledPrice',
 'F3_RolledPrice']:
    dataset[f'{name}_change'] = dataset[name] - dataset[name].shift(1)
    dataset[f'next_{name}_change'] =  dataset[name].shift(-1) - dataset[name] 
    
    

In [23]:
dataset.columns.tolist()

['tradeDate',
 'Name',
 'Commercial_NetPosition',
 'CommercialLongPosition',
 'CommercialShortPosition',
 'ManagedMoney_NetPosition',
 'ManagedMoney_LongPosition',
 'ManagedMoney_ShortPosition',
 'Commercial_NetPosition_change',
 'prior_report_Commercial_NetPosition_change',
 'CommercialLongPosition_change',
 'prior_report_CommercialLongPosition_change',
 'CommercialShortPosition_change',
 'prior_report_CommercialShortPosition_change',
 'ManagedMoney_NetPosition_change',
 'prior_report_ManagedMoney_NetPosition_change',
 'ManagedMoney_LongPosition_change',
 'prior_report_ManagedMoney_LongPosition_change',
 'ManagedMoney_ShortPosition_change',
 'prior_report_ManagedMoney_ShortPosition_change',
 'F1_RolledPrice',
 'F2_RolledPrice',
 'F3_RolledPrice',
 'F1_RolledPrice_rolling_20D_volatility',
 'F2_RolledPrice_rolling_20D_volatility',
 'F3_RolledPrice_rolling_20D_volatility',
 'SyntheticF1MinusF2_RolledPrice',
 'prior_report_SyntheticF1MinusF2_RolledPrice_change',
 'prior_cumulative_5D_F1_V

In [24]:
from research.model_selection.CombinatorialPurgedCV import CombinatorialPurgedCV
from research.model_selection.CombinatorialPurgedCV import cpcv_predict



In [25]:

from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor


In [26]:
?CombinatorialPurgedCV

Init signature: CombinatorialPurgedCV(n_partitions: int, k: int, purge_amount: int)
Docstring:     
Combinatorial Purged Cross-Validatpr with Purging

This class identifies training and testing indicies that split the data into train/test sets. It also implements purging to avoid lookahead bias.
Init docstring:
Args:
:param n_partitions: the total number of partitions
:param k: the number of partitions to include in the test set
:param purge_amount: the number of indices to be purged between train/test sets
File:           ~/Documents/Projects/omroot_repos/COTAME/research/model_selection/CombinatorialPurgedCV.py
Type:           type
Subclasses:     

In [27]:
response_names =  ['ManagedMoney_NetPosition_to_openinterest_change',
                'ManagedMoney_LongPosition_to_openinterest_change',
                'ManagedMoney_ShortPosition_to_openinterest_change'] 



In [28]:
cv = CombinatorialPurgedCV(n_partitions=10,
                           k=2,
                           purge_amount=10)

In [29]:
import json

In [30]:
with open("wti_cot_mm_selected_features_by_response.json", "r") as f:
    selected_features_by_response = json.load(f)

In [31]:
selected_features_by_response

{'ManagedMoney_NetPosition_to_openinterest_change': ['prior_report_ManagedMoney_NetPosition_change',
  'prior_report_ManagedMoney_LongPosition_change',
  'prior_5D_F1_OI_change',
  'prior_5D_AGG_OI_change',
  'F1_RolledPrice_change'],
 'ManagedMoney_LongPosition_to_openinterest_change': ['prior_report_ManagedMoney_ShortPosition_to_openinterest_change',
  'F3_RolledPrice_change',
  'prior_5D_F1_OI_change',
  'prior_5D_AGG_OI_change'],
 'ManagedMoney_ShortPosition_to_openinterest_change': ['prior_cumulative_5D_F2_Volume_change',
  'prior_report_ManagedMoney_NetPosition_change',
  'prior_report_ManagedMoney_LongPosition_change',
  'F1_RolledPrice_change']}

In [32]:
import json
with open("wti_cot_mm_selected_models.json", "r") as f:
    best_model_dict = json.load(f)

In [33]:
from src.preprocessing.base import Winsorizer

import numpy as np
import pandas as pd
from scipy.stats.mstats import winsorize
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import SGDRegressor, Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor

estimators_  = {
    # Elastic-net via SGD (fast, online)
    "enet": SGDRegressor(
        penalty="elasticnet",
        alpha=1e-4,
        l1_ratio=0.15,
        loss="squared_error",   # or "huber" for robustness
        max_iter=1000,
        tol=1e-3,
        random_state=42
    ),
    # Classic L1 / OLS / L2
    "lasso": Lasso(alpha=1.0, random_state=42),
    "ols": LinearRegression(),
    "ridge": Ridge(alpha=1.0, random_state=42),

    # Tree-based regressors
    "rf": RandomForestRegressor(n_estimators=250, max_depth=10, random_state=42),
    "extra": ExtraTreesRegressor(n_estimators=250, max_depth=10, random_state=42),
    "hgbm": HistGradientBoostingRegressor(max_iter=150, learning_rate=0.001, random_state=42),
}


In [34]:
final_models = {}
for (response_name, feature_names) in selected_features_by_response.items():
    best_model_name = best_model_dict[response_name]
    estimator = estimators_[best_model_name]
    Xy = dataset[['tradeDate']+feature_names+[response_name]].dropna()
    Xy.reset_index(drop=True, inplace= True)
    estimator.fit(Xy[feature_names], Xy[response_name])
    final_models[response_name] = estimator
    
    

In [35]:
final_models

{'ManagedMoney_NetPosition_to_openinterest_change': ExtraTreesRegressor(max_depth=10, n_estimators=250, random_state=42),
 'ManagedMoney_LongPosition_to_openinterest_change': RandomForestRegressor(max_depth=10, n_estimators=250, random_state=42),
 'ManagedMoney_ShortPosition_to_openinterest_change': ExtraTreesRegressor(max_depth=10, n_estimators=250, random_state=42)}

In [36]:
Settings.MODELS_DIR

PosixPath('/Users/oualid/Documents/Projects/omroot_repos/COTAME/cache/models')

In [37]:
import pickle

with open(Settings.MODELS_DIR / "wti_cot_mm_models.pkl", "wb") as f:
    pickle.dump(final_models, f)